In [1]:
import os
from tqdm.notebook import tqdm
import zipfile
import time
import skimage.io
import numpy as np
import pandas as pd
import cv2
import PIL.Image
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score
from tqdm import tqdm_notebook as tqdm

In [2]:
data_dir = ''
df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))

In [3]:
# Image ids in NAME variable with their respective classes in CLASS variable
NAME=['0005f7aaab2800f6170c399693a96917','000920ad0b612851f8e01bcc880d9b3d','003046e27c8ead3e3db155780dc5498e','0032bfa835ce0f43a92ae0bbab6871cb','00bbc1482301d16de3ff63238cfd0b34',
     '00d7ec94436e3a1416a3b302914957d3','0068d4c7529e34fd4c9da863ce01a161','006f6aa35a78965c92fffd1fbd53a058','0018ae58b01bdadc8e347995b69f99aa','001c62abd11fa4b57bf7a6c603a11bb9',
     '00928370e2dfeb8a507667ef1d4efcbb','00c15b23b30a5ba061358d9641118904']
CLASS=[0,0,1,1,2,2,3,3,4,4,5,5]

In [4]:
df=[]
count=0
for i in NAME:
 df.append(df_train[df_train['image_id']==i])
 count+=1
df_train=pd.DataFrame(np.array(df).reshape(12,4),columns=['image_id','data_provider','isup_grade','gleason_score'])
df_train

,image_id,data_provider,isup_grade,gleason_score
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0
2,003046e27c8ead3e3db155780dc5498e,karolinska,1,3+3
3,0032bfa835ce0f43a92ae0bbab6871cb,karolinska,1,3+3
4,00bbc1482301d16de3ff63238cfd0b34,radboud,2,3+4
5,00d7ec94436e3a1416a3b302914957d3,karolinska,2,3+4
6,0068d4c7529e34fd4c9da863ce01a161,radboud,3,4+3
7,006f6aa35a78965c92fffd1fbd53a058,radboud,3,4+3
8,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4
9,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4


In [14]:
TRAIN = 'train_images/'
MASKS = 'train_label_masks/'
OUT_TRAIN = 'zips/train.zip'
OUT_MASKS = 'zips/masks.zip'
sz = 256 # Size of each tile
N = 36 # Total no of tiles

In [15]:
def tile(img, mask):
    result = []
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=255)
    mask = np.pad(mask,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=0)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    mask = mask.reshape(mask.shape[0]//sz,sz,mask.shape[1]//sz,sz,3)
    mask = mask.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        mask = np.pad(mask,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=0)
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    img = img[idxs]
    mask = mask[idxs]
    for i in range(len(img)):
        result.append({'img':img[i], 'mask':mask[i], 'idx':i})
    return result

In [16]:
x_tot,x2_tot = [],[]
names = [name[:-10] for name in os.listdir(MASKS)]
with zipfile.ZipFile(OUT_TRAIN, 'w') as img_out,\
 zipfile.ZipFile(OUT_MASKS, 'w') as mask_out:
    for name in tqdm(NAME):
        img = skimage.io.MultiImage(os.path.join(TRAIN,name+'.tiff'))[-1]
        mask = skimage.io.MultiImage(os.path.join(MASKS,name+'_mask.tiff'))[-1]
        tiles = tile(img,mask)
        for t in tiles:
            img,mask,idx = t['img'],t['mask'],t['idx']
            x_tot.append((img/255.0).reshape(-1,3).mean(0))
            x2_tot.append(((img/255.0)**2).reshape(-1,3).mean(0)) 
            #if read with PIL RGB turns into BGR
            img = cv2.imencode('.png',cv2.cvtColor(img, cv2.COLOR_RGB2BGR))[1]
            img_out.writestr(f'{name}_{idx}.png', img)
            mask = cv2.imencode('.png',mask[:,:,0])[1]
            mask_out.writestr(f'{name}_{idx}.png', mask)

C:\Users\johng\AppData\Local\Temp\ipykernel_16016\1895463403.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for name in tqdm(NAME):


  0%|          | 0/12 [00:00<?, ?it/s]

In [18]:
! unzip 'zips/train.zip'

^C


In [19]:
# loading all the tile images into img_train array with 12 rows
import cv2
out_dir=''
img_train=np.empty((12,6*256,6*256,3))
count=-1
for i in NAME:
   img_var=[]
   count+=1
   for j in range(0,36): 
     img_var.append(cv2.imread(out_dir+str(i)+'_'+str(j)+'.png'))
   img_var=np.array(img_var)
   img_var=img_var.reshape(6*256,6*256,3)
   img_train[count]=img_var

In [20]:
img_train=img_train.reshape(12,-1) # for smote function to work, it requires only 2 dimensional array


In [25]:
# 12 to 24 images using smote analysis
from collections import Counter
from imblearn.over_sampling import SMOTE
counter = Counter(CLASS)
print('Before',counter)
# oversampling the train dataset using SMOTE
smt = SMOTE(sampling_strategy={0:4,1:4,2:4,3:4,4:4,5:4},k_neighbors=1)
#X_train, y_train = smt.fit_resample(X_train, y_train)
X_train_sm, y_train_sm = smt.fit_resample(img_train, CLASS)

counter = Counter(y_train_sm)
print('After',counter)

ImportError: cannot import name '_check_X' from 'imblearn.utils._validation' (C:\Users\johng\anaconda3\Lib\site-packages\imblearn\utils\_validation.py)

In [22]:
X_train=X_train_sm.reshape(24,6*256,6*256,3) # reshaping array after completion of smote analysis


NameError: name 'X_train_sm' is not defined